### Import Libraries

In [1]:
import pandas as pd
import re
import string
import nltk
import emoji
import csv
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from qalsadi import lemmatizer
from googletrans import Translator

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/madboly/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/pseudomar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/pseudomar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Load train data 

In [2]:
file_path = 'train.xlsx'
train = pd.read_excel(file_path)

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  1000 non-null   int64 
 1   review_description  1000 non-null   object
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [4]:
train.head()

,ID,review_description
0,1,اهنئكم على خدمه العملاء في المحادثه المباشره م...
1,2,ممتاز جدا ولكن اتمنى ان تكون هناك بعض المسابقا...
2,3,كل محملته يقول تم ايقاف حطيت2 عشان تسوون الخطاء
3,4,شغل طيب
4,5,بعد ماجربت


### Basic preprocessing

##### Checking for null values

In [5]:
train.isnull().sum()

ID                    0
review_description    0
dtype: int64

In [6]:
def remove_nulls_and_duplicates(df):
    df_cleaned = df.dropna()
    df_cleaned = df_cleaned.drop_duplicates()
    return df_cleaned

### Cleaning text

In [7]:
def remove_punctuation(text):
    punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation
    text = re.sub('[%s]' % re.escape(punctuations), ' ', text)
    return text

In [8]:
def remove_digits(text):
    return re.sub('\d+', '', text)

In [9]:
def remove_diacritics(text):
    return re.sub(r"[ًًٌٍَُِّْ]", "", text)

In [10]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("[يى]", "ي", text) 
    text = re.sub("[ؤئ]", "ء", text) 
    text = re.sub("ة", "ه", text)
    text = re.sub("ـ", "", text)
    text = re.sub("گ", "ك", text)
    return text

In [11]:
def remove_repeating_char(text):
    # Remove 3+ repeated consecutive characters
    return re.sub(r'(.)\1{2,}', r'\1', text)

In [12]:
def remove_long_words(text, threshold=15):
    return ' '.join(word for word in text.split(" ") if len(word) < threshold)

In [13]:
def remove_non_arabic_words(text):
    """
    Source: https://gist.github.com/mohabmes/33b724edfd4f0f3ec2e6644168db516e#file-preprocess_arabic_text-py-L22
    """
    return re.sub(r'[^\u0600-\u06ff\u0750-\u077f\ufb50-\ufbc1\ufbd3-\ufd3f\ufd50-\ufd8f\ufd50-\ufd8f\ufe70-\ufefc\uFDF0-\uFDFD]+', ' ', text)

#### Remove stopwords

In [14]:
stopwords_list = set(stopwords.words("english") + stopwords.words("arabic"))
words_to_keep = {'not', 'لا', 'ليس', 'مش', 'لم', 'لن', 'جدا', 'اكثر', 'قليل', 'كثير', 'حب', 'بكره', 'بحب', 'عجب', 'غير', 'كره'}
stopwords_list = stopwords_list.difference(words_to_keep)
len(stopwords_list)

874

In [15]:
def remove_stopwords(text):
    text = ' '.join([word for word in word_tokenize(text) if word not in stopwords_list])
    return text

#### Replace emojis

In [16]:
"""
Source:
https://github.com/a-ibrahimi/Arabic-Emojipedia
https://stackoverflow.com/a/76419165/13218954
"""
def build_emoji_dictionary():
    csv_file = 'emojis.csv'
    emoji_dict = {}
    with open(csv_file, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        for row in reader:
            emoji = row[0]
            text = row[1]
            emoji_dict[emoji] = text
    return emoji_dict

def replace_emojis(text):
    emoji_dict = build_emoji_dictionary()
    emojis = emoji.emoji_list(text)
    for emo in emojis:
        if emo['emoji'] in emoji_dict:
            # Replace the emoji with the corresponding text surrounded by spaces
            text = text.replace(emo['emoji'], ' ' + emoji_dict[emo['emoji']] + ' ')
    return text

#### Translate non-Arabic words

In [17]:
translator = Translator()

#### Lemmatization

In [18]:
arabic_lemmatizer = lemmatizer.Lemmatizer()

In [19]:
def lemmatize_word(word):
    try:
        language = translator.detect(word).lang
    except:
        return word
    
    if language == 'en':
        try:
            translated_word = translator.translate(word, src='en', dest='ar').text
            if translated_word in stopwords_list:
                # to be removed later
                return word
            word = translated_word
        except:
            return word

    try:
        lemma = arabic_lemmatizer.lemmatize(word)
        return lemma
    except:
        return word

def lemmatize_multilingual_text(text):
    lemmatized_words = [lemmatize_word(word) for word in text.split()]
    return ' '.join(lemmatized_words)

In [20]:
def preprocess(text):
    text = str(text).lower()
    text = remove_punctuation(text)
    text = remove_digits(text)
    text = remove_diacritics(text)
    text = normalize_arabic(text)
    text = remove_repeating_char(text)
    text = remove_long_words(text)
    text = remove_stopwords(text)

    text = replace_emojis(text)
    # Remove unhandled emojis/invalid characters
    text = re.sub(r'[^\w\s]','', text)
    # Collapse any consecutive spaces to a single space
    text = re.sub('\s+', ' ', text).strip()
    text = lemmatize_multilingual_text(text)
    # Remove any not-translated non-arabic word
    text = remove_non_arabic_words(text)
    # Normalize again as some words have inconcsistent data from lemmatization  
    text = remove_diacritics(text)
    text = normalize_arabic(text)   
    text = remove_stopwords(text)
 
    return text

## Clean train data

In [21]:
def clean_df(df):
    # utf-8 encoding
    df['preprocessed_review'] = df['review_description'].apply(lambda x: x.encode('utf-8').decode('utf-8'))
    # convert it to string
    df['preprocessed_review'] = df['preprocessed_review'].astype(str)
    
    df = remove_nulls_and_duplicates(df)
    df['preprocessed_review'] = df['preprocessed_review'].apply(preprocess)
    # A review may become empty after removing stopwords
    df = remove_nulls_and_duplicates(df)
    return df

In [34]:
train = clean_df(train)

In [22]:
train.head()

review_description  rating
0  شركه زباله و سواقين بتبرشم و مفيش حتي رقم للشك...      -1
1  خدمة الدفع عن طريق الكي نت توقفت عندي اصبح فقط...       1
2  تطبيق غبي و جاري حذفه ، عاملين اكواد خصم و لما...      -1
3  فعلا تطبيق ممتاز بس لو فى امكانية يتيح لمستخدم...       1
4  سيء جدا ، اسعار رسوم التوصيل لا تمت للواقع ب ص...      -1

In [ ]:
train.to_csv("preprocessed_train.csv")

## Clean test data

In [23]:
test = pd.read_csv('test.csv')

In [24]:
test.head()

,ID,review_description
0,1,اهنئكم على خدمه العملاء في المحادثه المباشره م...
1,2,ممتاز جدا ولكن اتمنى ان تكون هناك بعض المسابقا...
2,3,كل محملته يقول تم ايقاف حطيت2 عشان تسوون الخطاء
3,4,شغل طيب
4,5,بعد ماجربت


In [25]:
test = clean_df(test)

In [26]:
test.head()

,ID,review_description,preprocessed_review
0,1,اهنئكم على خدمه العملاء في المحادثه المباشره م...,اهنءكم علي خدم عملاء المحادثه المباشره قصر الل...
1,2,ممتاز جدا ولكن اتمنى ان تكون هناك بعض المسابقا...,ممتاز جدا اتمني ان تكون مسابق والجواءز طلبه ال...
2,3,كل محملته يقول تم ايقاف حطيت2 عشان تسوون الخطاء,محمل اقول تم ايقاف حطا عش تسوي خطاء
3,4,شغل طيب,شغل طيب
4,5,بعد ماجربت,ماجربت


In [27]:
test.to_csv("preprocessed_test.csv", index=False)